In [1]:
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('dark')

import warnings
warnings.filterwarnings('ignore')

# import torch
# from torch import nn
# from torchvision import models
# from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import platform
import sys
import os
import sklearn 
import cv2
import json
from glob import glob

print(f"- os: {platform.platform()}")
print(f"- python: {sys.version}")
print(f"- pandas: {pd.__version__}")
print(f"- numpy: {np.__version__}")
print(f"- sklearn: {sklearn.__version__}")
# print(f"- pytorch: {torch.__version__}")
print(f"- opencv: {cv2.__version__}")

- os: Windows-10-10.0.19041-SP0
- python: 3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]
- pandas: 1.3.0
- numpy: 1.19.1
- sklearn: 1.0.2
- opencv: 4.5.5


# csv분석

In [7]:
TRAIN_PATH = 'C:/git/jonie_github/VISION/_data/train/'
DATASET_NAME = 'CROP_DISEASE'
BATCH_SIZE = 128
EPOCHS = 150
USE_FOLD = True
SEED = 42
NUM_FOLD = 2

FLIP_IMG = True
BLUR_IMG = True
USE_AIHUB_DATA = False

# image resize
IMG_SIZE = 128

In [14]:
train_csv = sorted(glob(TRAIN_PATH +'*/*.csv'))
train_jpg = sorted(glob(TRAIN_PATH +'*/?????.jpg'))
train_json = sorted(glob(TRAIN_PATH +'*/*.json'))

test_csv = sorted(glob(TRAIN_PATH +'*/*.csv'))
test_jpg = sorted(glob(TRAIN_PATH +'*/*.jpg'))
test_json = sorted(glob(TRAIN_PATH +'*/*.json'))

In [15]:
crops = []
areas = []
diseases = []
risks = []
labels = []
img_names = []
parts=[]
objects = []
img_w = []
img_h = []
grows = []

for i in range(len(train_json)):
    with open(train_json[i], 'r') as f:
        sample = json.load(f)
        
        crop = sample['annotations']['crop']
        area = sample['annotations']['area']
        disease = sample['annotations']['disease']
        grow = sample['annotations']['grow']
        risk = sample['annotations']['risk']
        label=f"{crop}_{disease}_{risk}"        
        img_name = sample['description']['image'].replace('.jpg','')
        width = sample['description']['width']
        height = sample['description']['height']

        crops.append(crop)
        areas.append(area)
        diseases.append(disease)
        risks.append(risk)
        labels.append(label)
        img_names.append(img_name)
        img_w.append(width)
        img_h.append(height)
        grows.append(grow)
        r = sample['annotations']['bbox'][0]
        temp = [int(r['x']), int(r['y']), int(r['w']), int(r['h'])]
        objects.append(temp)    
        
        p = []
        
        for part in sample['annotations']['part']:
            p.append({
                'bbox': [
                    int(part['x']), int(part['y']), int(part['w']), int(part['h'])
                ],
                'id': part['id']
            })
        parts.append(p)

In [16]:
df = pd.DataFrame({'image':img_names, 'img_w':img_w, 'img_h':img_h, 'crops':crops,'areas':areas,'diseases':diseases,'risks':risks, 'grows':grows, 'bbox':objects, 'labels':labels, 'dir':train_jpg})
df['crop_area'] = df.crops.astype(str) + '_' + df.areas.astype(str)
df['disease_risk'] = df.diseases.astype(str) + '_' + df.risks.astype(str)

df['x'] = df.bbox.apply(lambda x: x[0])
df['y'] = df.bbox.apply(lambda x: x[1])
df['w'] = df.bbox.apply(lambda x: x[2])
df['h'] = df.bbox.apply(lambda x: x[3])
df['xc'] = df.x + df.w.apply(lambda x:x//2)
df['yc'] = df.y + df.h.apply(lambda x:x//2)
df.dir = df.dir.apply(lambda x: x.replace('\\','/'))

df = df[['image','crops','areas','diseases','risks','grows','xc','yc','w','h', 'img_w', 'img_h', 'dir']]
df

,image,crops,areas,diseases,risks,grows,xc,yc,w,h,img_w,img_h,dir
0,10027,3,3,b7,1,13,171,280,165,237,384,512,C:/git/jonie_github/VISION/_data/train/10027/1...
1,10037,3,5,00,0,11,190,159,48,315,384,512,C:/git/jonie_github/VISION/_data/train/10037/1...
2,10043,3,1,00,0,13,191,251,272,271,384,512,C:/git/jonie_github/VISION/_data/train/10043/1...
3,10045,3,3,00,0,11,214,319,172,263,384,512,C:/git/jonie_github/VISION/_data/train/10045/1...
4,10063,3,1,00,0,13,184,287,285,283,384,512,C:/git/jonie_github/VISION/_data/train/10063/1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5762,67640,3,5,00,0,11,131,185,160,370,384,512,C:/git/jonie_github/VISION/_data/train/67640/6...
5763,67644,1,3,00,0,11,401,258,560,488,682,512,C:/git/jonie_github/VISION/_data/train/67644/6...
5764,67647,2,3,a5,2,12,374,303,322,174,682,512,C:/git/jonie_github/VISION/_data/train/67647/6...
5765,67649,1,3,00,0,12,261,154,220,239,384,512,C:/git/jonie_github/VISION/_data/train/67649/6...


In [ ]:
# 1. dropduplicates
# 2. sort_values
# 3. 

In [211]:
csv_data[306].replace('0', np.nan)
csv_data[306].replace('-', np.nan).dropna()

,측정시각,내부 온도 1 평균,내부 온도 1 최고,내부 온도 1 최저,내부 습도 1 평균,내부 습도 1 최고,내부 습도 1 최저,내부 이슬점 평균,내부 이슬점 최고,내부 이슬점 최저
120,2021-09-28 14:00:00,32.9,40.0,28.1,37.4,63.3,0.0,20.1,20.3,20.1
119,2021-09-28 14:10:00,27.9,28.0,27.8,64.2,66.2,63.0,20.7,21.1,20.3
118,2021-09-28 14:20:00,27.6,27.8,27.4,64.7,65.7,63.9,20.7,20.8,20.1
117,2021-09-28 14:30:00,27.3,27.4,27.1,66.0,67.5,64.2,20.1,20.6,19.6
116,2021-09-28 14:40:00,26.9,27.0,26.9,66.5,66.9,65.5,20.3,20.4,20.1
...,...,...,...,...,...,...,...,...,...,...
4,2021-09-29 09:20:00,25.7,25.8,25.7,83.3,83.8,82.8,23.0,23.1,22.9
3,2021-09-29 09:30:00,25.6,25.7,25.6,84.3,85.4,83.4,23.2,23.3,22.9
2,2021-09-29 09:40:00,25.8,25.8,25.8,83.5,83.9,83.1,23.0,23.1,22.9
1,2021-09-29 09:50:00,25.9,25.9,25.8,83.3,84.4,82.9,22.9,23.1,22.9


In [217]:
from datetime import datetime, timedelta

csv_data = []
for csv_file in tqdm(train_csv):
    data = pd.read_csv(f'{csv_file}')
    data = data[['측정시각', '내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저','내부 습도 1 평균', '내부 습도 1 최고',
       '내부 습도 1 최저','내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']]
    data = data.drop_duplicates(subset=['측정시각'])
    data = data.sort_values(by='측정시각')
    data['측정시각'] = data['측정시각'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    data = data.replace('0', np.nan).dropna()
    data = data.replace('-', np.nan).dropna()
    csv_data.append(data)

100%|██████████████████████████████████████████████████████████████████████████████| 5767/5767 [02:38<00:00, 36.30it/s]


In [33]:
#포도 정상
csv_data[31]

,측정시각,내부 온도 1 평균,내부 온도 1 최고,내부 온도 1 최저,내부 습도 1 평균,내부 습도 1 최고,내부 습도 1 최저,내부 이슬점 평균,내부 이슬점 최고,내부 이슬점 최저
293,2021-09-12 08:30:00,23.3,23.6,22.9,85.7,86.4,84.6,20.7,21.5,20.4
292,2021-09-12 08:40:00,24.0,24.4,23.6,84.4,85.2,83.0,21.2,21.3,20.9
291,2021-09-12 08:50:00,25.0,25.6,24.4,81.3,82.4,79.4,21.5,22.1,20.7
290,2021-09-12 09:00:00,25.8,26.0,25.6,78.2,78.8,77.4,22.0,22.1,21.7
289,2021-09-12 09:10:00,26.0,26.1,25.8,76.4,77.4,75.7,21.6,21.7,21.5
...,...,...,...,...,...,...,...,...,...,...
4,2021-09-14 08:40:00,22.7,22.8,22.5,70.9,73.0,69.1,17.5,17.9,17.0
3,2021-09-14 08:50:00,22.5,22.6,22.4,71.6,72.9,70.1,17.6,17.9,17.0
2,2021-09-14 09:00:00,22.4,22.4,22.4,73.0,74.2,71.7,17.0,17.2,16.7
1,2021-09-14 09:10:00,22.7,22.9,22.4,72.4,73.4,71.2,17.7,17.9,17.0


In [126]:
#https://bmcnoldy.rsmas.miami.edu/Humidity.html
# RH = 100*(EXP((17.625*TD)/(243.04+TD))/EXP((17.625*T)/(243.04+T)))
# RH 상대습도 TD 이슬점 T 관측온도
# TD = csv_data[357]['내부 이슬점 평균']
# T = csv_data[357]['내부 온도 1 평균']
# RH = 100*(np.exp((17.625*TD)/(243.04+TD))/np.exp((17.625*T)/(243.04+T)))
# RH[RH>=100]

# TD = csv_data[31]['내부 이슬점 평균']
# T = csv_data[31]['내부 온도 1 평균']
# RH = 100*(np.exp((17.625*TD)/(243.04+TD))/np.exp((17.625*T)/(243.04+T)))
# RH[RH>100]

182    100.000000
181    101.247813
180    101.878268
179    102.513134
178    101.249726
177    101.249726
176    101.878268
175    100.621735
dtype: float64

In [232]:
 df_csv = pd.DataFrame(columns={'image','month','temp_cond_1','temp_cond_2','temp_cond_3','temp_cond_4',
                    'humidity_70','humidity_80','humidity_90','humidity_95','humidity_100',
                     'RH_70','RH_95','RH_100'})

for i, data in enumerate(tqdm(csv_data)):
    img_name = df.dir[i].split('/')[-1].replace('.jpg','')
    month = data.측정시각.dt.month.mean().astype(int)
    lenth = len(data)
    temp_cond_1 = len(data[(data['내부 온도 1 평균']>=20)&(data['내부 온도 1 평균']<=25)])/lenth
    temp_cond_2 = len(data[(data['내부 온도 1 최저']<=13)])/lenth
    temp_cond_3 = len(data[(data['내부 온도 1 최고']>=32) & (data['내부 온도 1 최고']<35)])/lenth
    temp_cond_4 = len(data[(data['내부 온도 1 최고']>=35)])/lenth

    humidity_70 = len(data[(data['내부 습도 1 평균']>=70)& (data['내부 습도 1 평균']<80)])/lenth
    humidity_80 = len(data[(data['내부 습도 1 평균']>=80)& (data['내부 습도 1 평균']<90)])/lenth
    humidity_90 = len(data[(data['내부 습도 1 평균']>=90)& (data['내부 습도 1 평균']<95)])/lenth
    humidity_95 = len(data[(data['내부 습도 1 평균']>=95)])/lenth
    
    TD = data['내부 이슬점 평균'].astype(float)
    T = data['내부 온도 1 평균'].astype(float)
    RH = 100*(np.exp((17.625*TD)/(243.04+TD))/np.exp((17.625*T)/(243.04+T)))

    RH_70 = len(RH[(RH>=70)& (RH<95)])/lenth
    RH_95 = len(RH[(RH>=95)& (RH<100)])/lenth
    RH_100 = len(RH[(RH>=100)])/lenth

    csv_series = pd.Series({'image':img_name, 'month':month,'temp_cond_1':temp_cond_1,'temp_cond_2':temp_cond_2,'temp_cond_3':temp_cond_3,'temp_cond_4':temp_cond_4,
                        'humidity_70':humidity_70,'humidity_80':humidity_80,'humidity_90':humidity_90,'humidity_95':humidity_95,
                         'RH_70':RH_70,'RH_95':RH_95,'RH_100':RH_100})
    df_csv.loc[i] = csv_series



100%|██████████████████████████████████████████████████████████████████████████████| 5767/5767 [01:48<00:00, 53.10it/s]


In [233]:
df_csv

,temp_cond_2,RH_100,temp_cond_3,humidity_70,humidity_100,RH_70,humidity_80,RH_95,temp_cond_4,image,temp_cond_1,month,humidity_90,humidity_95
0,0.727891,0.000000,0.000000,0.115646,NaN,0.755102,0.578231,0.000000,0.000000,10027,0.000000,11,0.064626,0.000000
1,0.000000,0.000000,0.000000,0.146259,NaN,0.642857,0.350340,0.064626,0.000000,10037,0.292517,9,0.210884,0.000000
2,0.000000,0.000000,0.000000,0.244898,NaN,0.741497,0.251701,0.027211,0.000000,10043,0.408163,6,0.282313,0.000000
3,0.000000,0.000000,0.000000,0.081633,NaN,0.632653,0.391156,0.003401,0.000000,10045,0.217687,9,0.163265,0.000000
4,0.000000,0.000000,0.000000,0.244898,NaN,0.741497,0.251701,0.027211,0.000000,10063,0.408163,6,0.282313,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5762,0.000000,0.000000,0.000000,0.095238,NaN,0.530612,0.190476,0.391156,0.000000,67640,0.707483,9,0.374150,0.251701
5763,0.000000,0.000000,0.003401,0.081633,NaN,0.571429,0.469388,0.000000,0.000000,67644,0.204082,9,0.010204,0.000000
5764,0.251701,0.040816,0.061224,0.139456,NaN,0.517007,0.159864,0.384354,0.037415,67647,0.214286,10,0.166667,0.472789
5765,0.000000,0.000000,0.006803,0.071429,NaN,0.588435,0.125850,0.081633,0.000000,67649,0.129252,10,0.459184,0.010204


In [158]:
# print(f'temp_cond_1:{temp_cond_1}')
# print(f'temp_cond_2:{temp_cond_2}')
# print(f'temp_cond_3:{temp_cond_3}')
# print(f'temp_cond_4:{temp_cond_4}')
# print(f'humidity_70:{humidity_70}')
# print(f'humidity_80:{humidity_80}')
# print(f'humidity_90:{humidity_90}')
# print(f'humidity_95:{humidity_95}')
# print(f'humidity_100:{humidity_100}')
# print(f'RH_70:{RH_70}')
# print(f'RH_95:{RH_95}')
# print(f'RH_100:{RH_100}')
# print(f'month:{month}')

# b= pd.Series({'month':month,'temp_cond_1':temp_cond_1,'temp_cond_2':temp_cond_2,'temp_cond_3':temp_cond_3,'temp_cond_4':temp_cond_4,
#                     'humidity_70':humidity_70,'humidity_80':humidity_80,'humidity_90':humidity_90,'humidity_95':humidity_95,'humidity_100':humidity_100,
#                      'RH_70':RH_70,'RH_95':RH_95,'RH_100':RH_100})

temp_cond_1:0.5136054421768708
temp_cond_2:0.0
temp_cond_3:0.1326530612244898
temp_cond_4:0.0
humidity_70:0.08163265306122448
humidity_80:0.17346938775510204
humidity_90:0.2619047619047619
humidity_95:0.09863945578231292
humidity_100:0.0
RH_70:0.5238095238095238
RH_95:0.06802721088435375
RH_100:0.027210884353741496
month:8


In [22]:
print(np.any([1,1,1]))
print(np.any([1,0,0,1]))

True
True


## 포도노균병
- 균사 생육온도 5∼21℃, 
- 분생포자 발아적온 **20∼22℃ **
- 발병조건 : 고온 다습
- 분생포자 형성 환경 : 기온 12℃, 습도 95∼100%
- 분생포자, 난포자 발아 적온 : 20∼25℃
- 주로 습기가 많고, 그늘지고, 통풍이 불량한 과원에서 많이 발병

## 포도탄저병
- 5월 월동 병반에서 분생포자 형성, 강우로 미숙과에서 분산, 전파 - 생육온도 7∼37℃, 발병적온 28℃
- 고온 다습(성숙기) 포도에 치명적인 피해

## 포도흰가루
- 고온건조(개화기부터 10월 상순)
- 최저 15℃, 최적 25∼30℃, 70%이상
- 특히 13℃ 이하의 저온, 32℃ 이상의 고온 병의 진전 둔화
- 5월 중순부터 발병하여 6∼7월에 많이 발병
- 특히 이른봄부터 여름사이에 기온이 높은 해에 심하게 발병

## 포도일소피해
- 과립온도 35℃이상이면 호흡이상으로 과실내 알데히드 축적, 세포가 죽음으로써 발생

## 포도 축과병
- 직사광선이 강하게 닿지 않는 과방의 속이나 아랫부분에 발생
- 장마기 또는 직후 온도 및 토양수분의 변화가 급격히 일어나 수분흡수와 증산의 균형이 깨져 일어남
- 비가 내린후 맑은 날씨가 되었을 때 심하게 발생


In [85]:
train_csv[0]

'C:/Users/jeong/Projects/dacon/병해진단/data/train\\10027\\10027.csv'

In [68]:
csv_data[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   측정시각        588 non-null    object 
 1   내부 온도 1 평균  588 non-null    float64
 2   내부 온도 1 최고  588 non-null    float64
 3   내부 온도 1 최저  588 non-null    float64
 4   내부 습도 1 평균  588 non-null    float64
 5   내부 습도 1 최고  588 non-null    float64
 6   내부 습도 1 최저  588 non-null    float64
 7   내부 이슬점 평균   588 non-null    float64
 8   내부 이슬점 최고   588 non-null    float64
 9   내부 이슬점 최저   588 non-null    float64
dtypes: float64(9), object(1)
memory usage: 46.1+ KB


In [69]:
csv_data[0]

,측정시각,내부 온도 1 평균,내부 온도 1 최고,내부 온도 1 최저,내부 습도 1 평균,내부 습도 1 최고,내부 습도 1 최저,내부 이슬점 평균,내부 이슬점 최고,내부 이슬점 최저
0,2021-11-26 09:30:00,9.0,9.6,8.5,81.5,82.0,80.8,6.1,6.9,5.9
1,2021-11-26 09:30:00,8.7,9.3,8.1,85.4,85.7,84.9,6.3,6.8,5.7
2,2021-11-26 09:20:00,7.9,8.4,7.4,81.9,82.7,81.6,5.0,5.1,4.3
3,2021-11-26 09:20:00,7.4,7.9,6.9,86.9,87.4,85.9,5.4,6.0,5.0
4,2021-11-26 09:10:00,6.9,7.2,6.6,83.3,84.0,82.7,4.4,4.5,4.3
...,...,...,...,...,...,...,...,...,...,...
583,2021-11-24 09:00:00,8.7,8.9,8.4,89.7,90.1,89.2,7.3,7.5,6.5
584,2021-11-24 08:50:00,8.8,9.0,8.7,85.3,85.5,85.0,6.6,6.8,6.6
585,2021-11-24 08:50:00,8.2,8.4,8.0,89.9,90.1,89.6,6.5,6.5,6.5
586,2021-11-24 08:40:00,8.5,8.6,8.4,85.5,85.8,85.3,6.4,6.8,5.8


In [75]:
temp = csv_data[0].copy()
temp['ew'] = temp['내부 온도 1 평균']

,측정시각,내부 온도 1 평균,내부 온도 1 최고,내부 온도 1 최저,내부 습도 1 평균,내부 습도 1 최고,내부 습도 1 최저,내부 이슬점 평균,내부 이슬점 최고,내부 이슬점 최저
586,2021-11-24 08:40:00,8.5,8.6,8.4,85.5,85.8,85.3,6.4,6.8,5.8
584,2021-11-24 08:50:00,8.8,9.0,8.7,85.3,85.5,85.0,6.6,6.8,6.6
582,2021-11-24 09:00:00,9.3,9.5,9.1,85.3,85.8,84.6,6.8,7.6,6.6
580,2021-11-24 09:10:00,9.7,9.9,9.5,83.8,84.5,83.0,7.5,7.6,7.3
578,2021-11-24 09:20:00,10.0,10.1,9.9,82.4,83.0,82.0,7.2,7.3,7.1
...,...,...,...,...,...,...,...,...,...,...
8,2021-11-26 08:50:00,5.3,5.5,5.1,88.7,88.8,88.5,3.4,4.3,3.3
6,2021-11-26 09:00:00,6.4,6.6,6.2,83.8,84.2,83.6,3.7,4.5,3.5
4,2021-11-26 09:10:00,6.9,7.2,6.6,83.3,84.0,82.7,4.4,4.5,4.3
2,2021-11-26 09:20:00,7.9,8.4,7.4,81.9,82.7,81.6,5.0,5.1,4.3


In [ ]:
crop_dict = {1:'딸기', 
             2:'토마토',
             3:'파프리카',
             4:'오이', 
             5:'고추',
             6: '시설포도'}

area_dict = {1:'열매',
             2:'꽃',
             3:'잎',
             4:'가지',
             5:'줄기',
             6:'뿌리',
             7:'해충'}

task_dict = {0:'정상',
             1:'병해',
             2:'생리장애',
             3:'보호제처리반응'}

disease_dict = {'00' : '정상',
                
                # 병해 a
                'a1' : '딸기잿빛곰팡이병',
                'a2' : '딸기흰가루병',
                'a3' : '오이노균병',
                'a4' : '오이흰가루병',
                'a5' : '토마토흰가루병',
                'a6' : '토마토잿빛곰팡이병',
                'a7' : '고추탄저병',
                'a8' : '고추흰가루병',
                'a9' : '파프리카흰가루병',
                'a10' : '파프리카잘록병',
                'a11' : '시설포도탄저병',
                'a12' : '시설포도노균병',
                
                # 생리장애 b
                'b1' : '냉해피해',
                'b2' : '열과',
                'b3' : '칼슘결핍',
                'b4' : '일소피해',
                'b5' : '축과병',
                'b6' : '다량원소결핍 (N)',
                'b7' : '다량원소결핍 (P)',
                'b8' : '다량원소결핍 (K)',
                
                # 보호제 처리반응 c
                'c1' : '딸기잿빛곰팡이병반응',
                'c2' : '딸기흰가루병반응',
                'c3' : '오이노균병반응',
                'c4' : '오이흰가루병반응',
                'c5' : '토마토흰가루병반응',
                'c6' : '토마토잿빛곰팡이병반응',
                'c7' : '고추탄저병반응',
                'c8' : '고추흰가루병반응',
                'c9' : '파프리카흰가루병반응',
                'c10' : '파프리카잘록병반응',
                'c11' : '시설포도탄저병반응',
                'c12' : '시설포도노균병반응'
               }

            # 시설 과채류
grow_dict = {11:'유묘기',
             12:'생장기',
             13:'착화/과실기',
             
             # 시설 과수류 (포도)
             21:'발아기',
             22:'개화기',
             23:'신초생장기',
             24:'과실성숙기',
             25:'수확기',
             26:'휴면기'}

risk_dict = {0:'정상',
             1:'초기',
             2:'중기',
             3:'말기'}